# YOLOv10n

In [2]:
import os
import sys
import subprocess
import urllib.request
import zipfile
from pathlib import Path

Install required packages

In [ ]:
def install_requirements():
    packages = [
        "ultralytics",
        "torch",
        "torchvision",
        "tensorflow",
        "onnx",
        "onnx2tf",
        "onnxsim",
    ]

    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    print("Package installed successfully.")

Check for existing dataset or download COCO128

In [ ]:
def prepare_dataset():
    # Add paths to check for existing datasets, change the path based on your environment (Google Colab, local, etc.)
    potential_datasets = [  
        "/content/dataset",
        "/content/coco128"
    ]
    # Check for existing datasets
    for dataset_path in potential_datasets:
        if os.path.exists(dataset_path):
            # Look for YOLO dataset structure (images and labels folders)
            images_dir = os.path.join(dataset_path, 'images')
            labels_dir = os.path.join(dataset_path, 'labels')

            # Also check for common YOLO yaml files
            yaml_files = [f for f in os.listdir(dataset_path) if f.endswith('.yaml') or f.endswith('.yml')]

            if os.path.exists(images_dir) and os.path.exists(labels_dir):
                # Check if folders have content
                has_images = len([f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]) > 0
                has_labels = len([f for f in os.listdir(labels_dir) if f.lower().endswith('.txt')]) > 0

                if has_images and has_labels:
                    print(f"Existing dataset found at: {dataset_path}")

                    # Try to find or create a yaml file
                    yaml_path = None
                    if yaml_files:
                        yaml_path = os.path.join(dataset_path, yaml_files[0])
                        print(f"Use existing yaml file: {yaml_files[0]}")
                    else:
                        # Create a basic yaml file
                        yaml_path = create_basic_yaml(dataset_path)

                    return dataset_path, yaml_path
    # No existing dataset found, download COCO128
    print("No existing dataset found. Downloading COCO128.")
    coco128_path = download_coco128()
    yaml_path = os.path.join(coco128_path, 'coco128.yaml')

    return coco128_path, yaml_path

Create basic YAML


In [ ]:
def create_basic_yaml(dataset_path):
    yaml_content = f"""
        # Dataset configuration
        path: {dataset_path}
        train: images
        val: images

        # Classes (will be auto-detected from labels)
        nc: 80  # number of classes (placeholder)
        names: ['class0', 'class1', 'class2']  # class names (placeholder)
    """
    yaml_path = os.path.join(dataset_path, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)

    print(f"Created basic yaml configuration: {yaml_path}")
    return yaml_path

Download Coco128

In [ ]:
def download_coco128():
    dataset_url = "https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip"
    dataset_path = "/content/coco128.zip"
    extract_path = "/content/"

    print("Download COCO128 dataset")
    urllib.request.urlretrieve(dataset_url, dataset_path)

    print("Extract dataset")
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Remove zip file to save space
    os.remove(dataset_path)

    # Verify the yaml file exists and fix path if needed
    coco128_dir = "/content/coco128"
    yaml_file = os.path.join(coco128_dir, "coco128.yaml")

    if not os.path.exists(yaml_file):
        print("coco128.yaml not found. Checking for alternative locations.")
        # Look for any yaml files in the directory
        yaml_files = [f for f in os.listdir(coco128_dir) if f.endswith('.yaml') or f.endswith('.yml')]
        if yaml_files:
            print(f"Found yaml file: {yaml_files[0]}")
        else:
            print("No yaml file found. Creating basic configuration...")
            create_coco128_yaml(coco128_dir)

    print("COCO128 dataset downloaded and extracted successfully!")
    return coco128_dir

Add Coco128 YAML

In [ ]:
def create_coco128_yaml(dataset_path):
    yaml_content = f"""# COCO128 dataset configuration
        path: {dataset_path}
        train: images/train2017
        val: images/train2017

        # Classes
        nc: 80
        names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
            'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
            'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
            'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
            'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
            'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
            'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
            'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
            'hair drier', 'toothbrush']
    """

    yaml_path = os.path.join(dataset_path, 'coco128.yaml')
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)

    print(f"Created coco128.yaml at: {yaml_path}")

# TRAIN YOLOv10n

In [ ]:
def train_yolov10n(yaml_path, epochs=100, imgsz=640):
    from ultralytics import YOLO

    print("Initializing YOLOv10n model.")
    model = YOLO('yolov10n.pt')  # Load pretrained YOLOv10n model

    print(f"Starting training for {epochs} epochs.)
    print(f"Using dataset configuration: {yaml_path}")

    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=16,
        device=0 if os.system('nvidia-smi') == 0 else 'cpu',  # Use GPU if available
        project='/content/yolov10_training',
        name='training_experiment',
        save=True,
        cache=True,
        verbose=True
    )

    print("Training completed!")
    return model, results

# EXPORT MODEL

In [ ]:
def export_model_formats(model, export_dir="/content/exported_models"):
    os.makedirs(export_dir, exist_ok=True)

    print("Exporting model to different formats.")

    # Export to ONNX (intermediate format for TensorFlow conversion)
    onnx_path = model.export(format='onnx', dynamic=False, simplify=True)
    print(f"ONNX model exported to: {onnx_path}")

    # Export to TensorFlow SavedModel format
    try:
        tf_path = model.export(format='saved_model')
        print(f"TensorFlow SavedModel exported to: {tf_path}")
    except Exception as e:
        print(f"Direct TensorFlow export failed: {e}")
        tf_path = None

    return onnx_path, tf_path

Tflite Conversion

In [ ]:
def convert_to_tflite(onnx_path, output_dir="/content/tflite_models"):
    import tensorflow as tf

    os.makedirs(output_dir, exist_ok=True)

    try:
        # First convert ONNX to TensorFlow SavedModel using onnx2tf
        print("Converting ONNX to TensorFlow SavedModel.")
        saved_model_dir = f"{output_dir}/saved_model"

        # Use onnx2tf for conversion
        cmd = f"onnx2tf -i {onnx_path} -o {saved_model_dir} --non_verbose"
        subprocess.run(cmd, shell=True, check=True)

        # Convert TensorFlow SavedModel to TensorFlow Lite
        print("Converting TensorFlow SavedModel to TensorFlow Lite.")
        converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

        # Optimization settings for mobile deployment
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]  # Use FP16 for smaller size

        # Convert the model
        tflite_model = converter.convert()

        # Save the TensorFlow Lite model
        tflite_path = f"{output_dir}/yolov10n_coco128.tflite"
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)

        print(f"TensorFlow Lite model saved to: {tflite_path}")

        # Get model size
        model_size = os.path.getsize(tflite_path) / (1024 * 1024)  # Size in MB
        print(f"TensorFlow Lite model size: {model_size:.2f} MB")

        return tflite_path

    except Exception as e:
        print(f"TensorFlow Lite conversion failed: {e}")
        return None

Validate Tflite model

In [11]:
def validate_tflite_model(tflite_path, test_image_path=None):
    import tensorflow as tf
    import numpy as np

    # Load TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print("TensorFlow Lite Model Validation:")
    print(f"Input shape: {input_details[0]['shape']}")
    print(f"Input type: {input_details[0]['dtype']}")
    print(f"Number of outputs: {len(output_details)}")

    for i, output in enumerate(output_details):
        print(f"Output {i} shape: {output['shape']}")
        print(f"Output {i} type: {output['dtype']}")

    # Test with random input if no test image provided
    if test_image_path is None:
        input_shape = input_details[0]['shape']
        test_input = np.random.random(input_shape).astype(np.float32)

        interpreter.set_tensor(input_details[0]['index'], test_input)
        interpreter.invoke()

        print("Test inference completed successfully!")

        # Get outputs
        for i, output in enumerate(output_details):
            output_data = interpreter.get_tensor(output['index'])
            print(f"Output {i} min/max: {output_data.min():.4f}/{output_data.max():.4f}")

    return True


# MAIN

In [ ]:
def main():
    try:
        # Install requirements
        print("Installing required packages.")
        install_requirements()

        # Check for existing dataset or prepare COCO128
        print("Checking for existing dataset.")
        dataset_path, yaml_path = prepare_dataset()

        # Train model
        print("Training YOLOv10n model.")
        model, results = train_yolov10n(yaml_path, epochs=50)  # Reduced for faster training

        # Export model formats
        print("Exporting model to different formats.")
        onnx_path, tf_path = export_model_formats(model)

        # Convert to TensorFlow Lite
        print("Converting to TensorFlow Lite.")
        tflite_path = convert_to_tflite(onnx_path)

        if tflite_path:
            # Validate TFLite model
            print("Validating TensorFlow Lite model.")
            validate_tflite_model(tflite_path)
        else:
            print("Program finished with errors during TFLite conversion.")

    except Exception as e:
        print(f"Pipeline failed with error: {e}")
        import traceback
        traceback.print_exc()

# Simple TFLITE Conversion

In [ ]:
def simple_tflite_conversion():
    from ultralytics import YOLO

    print("Running simplified TensorFlow Lite conversion.")

    # Load pre-trained YOLOv10n
    model = YOLO('yolov10n.pt')

    # Direct export to TensorFlow Lite (if supported)
    try:
        tflite_path = model.export(format='tflite', imgsz=640)
        print(f"Direct TFLite export successful: {tflite_path}")
        return tflite_path
    except Exception as e:
        print(f"Direct TFLite export failed: {e}")
        return None

In [ ]:
if __name__ == "__main__":
    # Run in Google Colab environment
    print("Starting YOLOv10n training and TensorFlow Lite conversion.")

    main()

    # Simple test conversion
    simple_tflite_conversion()

Starting YOLOv10n training and TensorFlow Lite conversion...
Note: This process may take 1-2 hours depending on training epochs and hardware.
=== YOLOv10n Training and TensorFlow Lite Conversion Pipeline ===

1. Installing required packages...
All required packages installed successfully!

2. Checking for existing dataset...
No existing dataset found. Downloading COCO128...
Extracting dataset...
coco128.yaml not found, checking for alternative locations...
No yaml file found, creating basic configuration...
Created coco128.yaml at: /content/coco128/coco128.yaml
COCO128 dataset downloaded and extracted successfully!

3. Training YOLOv10n model...
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Initializing YOLOv10n model...
Star